In [2]:
import torch

In [3]:
torch.cuda.is_available()

True

In [4]:
import pandas as pd
df = pd.read_csv(r"E:\Projects\encoder - Decoder - chatbot\qajokes1.1.2.csv")
df.head(5)

,Question,Answer,dirt,humor,kwid,meta
0,What's the best anti diarrheal prescription?,Mycheexarphlexin,0.0,0.2,BEST-ANTI-DIARRHEAL-PRESCRIPTION-MYCHEEXARPHLEXIN,NaN
1,What do you call a person who is outside a doo...,Matt,0.0,0.2,PERSON-OUTSIDE-DOOR-ARMS-LEGS-MATT,NaN
2,Which Star Trek character is a member of the m...,Jean-Luc Pickacard,0.0,0.2,STAR-TREK-CHARACTER-MEMBER-MAGIC-CIRCLE-JEANLU...,NaN
3,What's the difference between a bullet and a h...,A bullet doesn't miss Harambe,0.0,0.2,DIFFERENCE-BULLET-HUMAN-BULLET-DOESNT-MISS-HAR...,NaN
4,Why was the Ethiopian baby crying?,He was having a mid-life crisis,0.0,0.2,ETHIOPIAN-BABY-CRYING-MIDLIFE-CRISIS,NaN


In [5]:
df = df.drop(columns=['kwid' , 'meta', 'dirt' , 'humor'])
df = df.dropna()
df = df.sample(frac=1).reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75107 entries, 0 to 75106
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Question  75107 non-null  object
 1   Answer    75107 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [6]:
df.sample(5)

,Question,Answer
41819,What do you do if you're attacked by a bunch o...,Go for the Jugular (juggler)!
22310,Why is owning a Prius difficult?,It's hard to drive when you're patting yoursel...
23309,Why did the jellybean go to school?,A. Because he wanted to be a smarty
33058,Who's there ! Bunny ! Bunny who ?,Bunny thing is I've forgotten now !kn
2115,Why wasn't the joke funny at the dance?,There wasn't a punch line.


In [7]:
#combine convokit dataset also
df2 = pd.read_csv(r"E:\Projects\encoder - Decoder - chatbot\Convokit_dataset.csv")
df2 = df2.drop(columns=['Unnamed: 0'])
df2 = df2.dropna()
df2 = df2.sample(frac=1).reset_index(drop=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221282 entries, 0 to 221281
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   questions  221282 non-null  object
 1   answers    221282 non-null  object
dtypes: object(2)
memory usage: 3.4+ MB


In [8]:
#concatinating both datsets
# Rename columns in df2 to match df
df2 = df2.rename(columns={'questions': 'Question', 'answers': 'Answer'})

# Concatenate both datasets
dfn = pd.concat([df, df2], ignore_index=True)

dfn = dfn.sample(frac=1).reset_index(drop=True)

dfn.head()

,Question,Answer
0,What's the difference between Navy SEAL traini...,read aloud) One is six months of hell with a t...
1,I hate waiting. I could give you my word as a ...,That's very comforting. But I'm afraid you'll ...
2,"Goodbye, Betty.",Not sometimes. Always. Just because it is th...
3,What does an Indian kid say before leaving his...,Mumbai
4,I don't understand...,I belong here.


In [9]:
dfn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296389 entries, 0 to 296388
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Question  296389 non-null  object
 1   Answer    296389 non-null  object
dtypes: object(2)
memory usage: 4.5+ MB


Tokenization

In [10]:
PAD_token = 0 #used for padding short sentences
SOS_token = 1 #start of sentence token
EOS_token = 2 #End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD" , SOS_token:"SOS", EOS_token:"EOS"}
        self.num_words = 3 #count sos , eos , pad

    def addSentence(self , sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self , word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word]=1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] +=1

In [11]:
import unicodedata
import re

# Define contractions dictionary
good_prefixes = {
    "i'm": "i am",
    "he's": "he is",
    "she's": "she is",
    "it's": "it is",
    "that's": "that is",
    "what's": "what is",
    "where's": "where is",
    "who's": "who is",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "can't": "cannot",
    "won't": "will not",
    "isn't": "is not",
    "aren't": "are not",
    "wasn't": "was not",
    "weren't": "were not",
    "i'll": "i will",
    "you'll": "you will",
    "we'll": "we will",
    "they'll": "they will",
}

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = " ".join([good_prefixes[word] if word in good_prefixes else word for word in s.split()])
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()    
    return s

# Read query / response pairs and return a voc object
def readVocs(datafile , corpus_name):
    print("Reading lines......")
    
    # Read the file and split into lines
    lines = open(datafile , encoding='utf-8').read().strip().split('\n')

    #split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('<CoSe')] for l in lines]
    voc = Voc(corpus_name)
    return voc , pairs


# Returns true iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
MAX_LENGTH = 15
def filterPair(p):
    #input sequences need to preserve the last word for EOS token 
    return len(p[0].split(' '))<MAX_LENGTH and len(p[1].split(' '))<MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

#using the functions defined above , return a populated voc object and pairs list
def loadPrepareData(corpus_name , datafile , save_dir):
    print("Start preparing training data .... ")
    voc , pairs = readVocs(datafile , corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words....")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words: ", voc.num_words)
    return voc , pairs

Batchwise input and data preparation

In [12]:
import itertools

PAD_token = 0  # used for padding short sentences
SOS_token = 1  # start of sentence token
EOS_token = 2  # End-of-sentence token

def indexesFromSentence(voc , sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]

def zeroPadding(l , fillvalue = PAD_token):
    return list(itertools.zip_longest(*l , fillvalue = fillvalue))

def binaryMatrix(l , value=PAD_token):
    m = []
    for i , seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# returns padded input sequence tensor and lengths 
def inputVar(l , voc):
    indexes_batch = [indexesFromSentence(voc , sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar , lengths

# returns padded target sequence tensor, padding mask , and max target length
def outputVar(l , voc):
    indexes_batch = [indexesFromSentence(voc , sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar , mask , max_target_len

#returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
        print("Input sentence-",pair[0])
        print("Output sentece-",pair[1])
        print("-----------------------")
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


In [13]:
import random

# Create a vocabulary instance
voc = Voc('test')

# Create pairs from your existing dataframe and normalize them
print("Creating pairs from dataframe...")
pairs = []
for _, row in dfn.iterrows():
    question = normalizeString(row['Question'])
    answer = normalizeString(row['Answer'])
    pairs.append([question, answer])

# Build vocabulary from pairs
print("Building vocabulary...")
for pair in pairs:
    voc.addSentence(pair[0])
    voc.addSentence(pair[1])

# Example for validation
print("\nTesting with a small batch...")
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("\nBatch information:")
print("Input shape:", input_variable.shape)
print("Lengths:", lengths)
print("Output shape:", target_variable.shape)

Creating pairs from dataframe...
Building vocabulary...

Testing with a small batch...
Input sentence- we re thinking of doing this current affairs thing for schools . world history since . twelve programmes lots of stock film .
Output sentece- and you re so beautiful . what are you doing here ?
-----------------------
Input sentence- a ha ! you landed on candyland ! now swallow it !
Output sentece- if he heard you talking like that he d make you walk the plank . three two one . now it is your turn spin .
-----------------------
Input sentence- why do mules not work as hard as horses ?
Output sentece- because they re half assed !
-----------------------
Input sentence- what drink can wrongly convict a black man ?
Output sentece- tequila mockingbird
-----------------------
Input sentence- there s not much room
Output sentece- oh no honey i will do that . . .
-----------------------

Batch information:
Input shape: torch.Size([24, 5])
Lengths: tensor([24, 13, 11, 10,  6])
Output shape: t

In [14]:
def filterPair(voc, pair):
	# Return True if all words in both sentences are in vocab
	return all(word in voc.word2index for word in pair[0].split()) and \
		   all(word in voc.word2index for word in pair[1].split())

# Get valid pairs
valid_pairs = [pair for pair in pairs if filterPair(voc, pair)]

small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(valid_pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("\nInput variable:")
print("input shape:", input_variable.shape)
print("lengths:", lengths)
print("\nTarget variable:")
print("Output shape:", target_variable.shape)
print("mask shape:", mask.shape)
print("max_target_len:", max_target_len)

Input sentence- how do i know this is not just some kind of you know seduction or something ?
Output sentece- sometimes . when it is convenient .
-----------------------
Input sentence- he was a very wealthy man he looked after me since i was young .
Output sentece- was he a relative ? or an intimate friend ?
-----------------------
Input sentence- where the music s playing and the ladies are . . .
Output sentece- in the yellow room where the music s playing and the ladies are ?
-----------------------
Input sentence- why is it unjust to blame taxi drivers for cheating us ?
Output sentece- we call them to take us in . from the project gutenberg ebook of the handbook of conundrums by edith b . ordway . so this is a century old joke .
-----------------------
Input sentence- what does he weigh ?
Output sentece- meat hes a butcher .
-----------------------

Input variable:
input shape: torch.Size([18, 5])
lengths: tensor([18, 16, 13, 13,  6])

Target variable:
Output shape: torch.Size([33,

Architecture

In [15]:
import torch.nn as nn
class EncoderRNN(nn.Module):
    def __init__(self , hidden_size, embedding, n_layers=1,dropout=0):
        super(EncoderRNN,self).__init__()
        self.hidden_size = hidden_size
        self.embedding = embedding

#Initialize GRU ; THE input_size and hidden_size params are both set to 'hidden_size'ArithmeticError
#because our input size is a word embedding with number of features == hidden_size

        self.gru = nn.GRU(hidden_size,hidden_size,n_layers, dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths , hidden=None):
        
        #convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        print(f"Shape after embedding: {embedded.shape}")
        
        #pack padded batch of sequences for RNN modu;e
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        print(f"Shape after packing: {packed.data.shape}")
        
        #forward pass through GRU
        outputs , hidden = self.gru(packed , hidden)
        print(f"Shape after GRU: {outputs.data.shape}")
        
        #unpack padding
        outputs , _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        print(f"Shape after unpacking: {outputs.shape}")
        
        #sum birectional GRU  outputs
        outputs = outputs[:,:,:self.hidden_size]+outputs[:,:,self.hidden_size]+outputs[:,:,self.hidden_size:]
        print(f"Shape after summing bidirectional outputs: {outputs.shape}")
        
        #return output and final hidden state
        return outputs, hidden

Attention layer and decoder

attention layer

In [16]:
import torch.nn.functional as F

# Luong attention layer
class Attn(torch.nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = torch.nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = torch.nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = torch.nn.Parameter(torch.FloatTensor(hidden_size))
def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)
def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)
def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)
def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)
# Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()
# Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

decoder

In [17]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()
# Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout
# Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.attn = Attn(attn_model, hidden_size)
def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden